In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import ast
import time
import matplotlib.pyplot as plt

# Step 8

We now create data for plotting ttc so we can explore TTC feature with histograms and maps.

In [2]:
# Specify the directory to load files from
directory = r'C:\Users\yftac\Documents\00_Project_Afeka\ttc_calculation\07_data_with_ttc'
# Get a list of files in the directory
file_name_data_ttc_list = os.listdir(directory)
file_name_data_ttc_list

['18_data_with_ttc.csv',
 '19_data_with_ttc.csv',
 '20_data_with_ttc.csv',
 '21_data_with_ttc.csv',
 '22_data_with_ttc.csv',
 '23_data_with_ttc.csv',
 '24_data_with_ttc.csv',
 '25_data_with_ttc.csv',
 '26_data_with_ttc.csv',
 '27_data_with_ttc.csv',
 '28_data_with_ttc.csv',
 '29_data_with_ttc.csv']

In [3]:
recordingId_list = file_name_data_ttc_list.copy()
for i in range(len(recordingId_list)):
    recordingId_list[i]=recordingId_list[i].replace('_data_with_ttc.csv', '')
recordingId_list

['18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29']

In [4]:
root_dir = r"C:\Users\yftac\Documents\00_Project_Afeka\ttc_calculation"
folder_name='07_data_with_ttc'

# Load dataframes
def load_dfs(file_name_list, root_dir, folder_name):
    os.chdir(root_dir)
    # change to data dir
    os.chdir(folder_name)
    dfs = []
    for i in range(len(file_name_list)):
        df=pd.read_csv(file_name_list[i])
        dfs.append(df)
    # change back to parent directory
    os.chdir('..')
    return dfs

dfs = load_dfs(file_name_data_ttc_list, root_dir, folder_name)

In [5]:
# Print the length of each DataFrame in the dfs list
for i, df in enumerate(dfs):
    print(f"Length of DataFrame {i}: {len(df)}")

Length of DataFrame 0: 42378
Length of DataFrame 1: 33777
Length of DataFrame 2: 43601
Length of DataFrame 3: 54919
Length of DataFrame 4: 56011
Length of DataFrame 5: 53853
Length of DataFrame 6: 44711
Length of DataFrame 7: 55189
Length of DataFrame 8: 60034
Length of DataFrame 9: 65115
Length of DataFrame 10: 56613
Length of DataFrame 11: 67337


In [6]:
# print dfs info
def print_dfs_info(dfs, file_name_data_ttc_list):
    for i in range(len(dfs)):
        print(f'{file_name_data_ttc_list[i]} df:\n')
        print(dfs[i].info())
        print('------------------------------------------------------\n')

print_dfs_info(dfs, file_name_data_ttc_list)

18_data_with_ttc.csv df:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42378 entries, 0 to 42377
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   recordingId    42378 non-null  int64  
 1   trackId        42378 non-null  int64  
 2   frame          42378 non-null  int64  
 3   trackLifetime  42378 non-null  int64  
 4   xCenter        42378 non-null  float64
 5   yCenter        42378 non-null  float64
 6   heading        42378 non-null  float64
 7   width          42378 non-null  float64
 8   length         42378 non-null  float64
 9   xVelocity      42378 non-null  float64
 10  yVelocity      42378 non-null  float64
 11  class          42378 non-null  object 
 12  pairs          42378 non-null  object 
 13  num_of_pairs   42378 non-null  int64  
 14  hx             42378 non-null  float64
 15  hy             42378 non-null  float64
dtypes: float64(9), int64(5), object(2)
memory usage: 5.2+ MB
None
------

Create a empty df that has in same row 2 pair objects to plot on map

In [7]:
df_ttc = pd.DataFrame(columns=[
    'recordingId', 'trackId_pedestrian', 'xCenter_pedestrian', 'yCenter_pedestrian', 
    'class_pedestrian', 'frame', 
    'trackId_peer', 'xCenter_peer', 'yCenter_peer', 
    'class_peer', 'frame_peer', 
    'ttc'])
# new keys for pedestrian i to be created
keys_i = ['recordingId', 'trackId_pedestrian', 'xCenter_pedestrian', 'yCenter_pedestrian', 
          'class_pedestrian','frame']    
# new keys for peer n_j to be created
keys_j_n = ['trackId_peer','xCenter_peer', 'yCenter_peer',
            'class_peer', 'frame_peer'] 

In [8]:
# this function creates new df that has pedestrian-peer info for plotting
def create_df_series_for_plot(df_temp, df_ttc, keys_i, keys_j_n):   
    for i in tqdm(range(len(df_temp)), desc='Creating DataFrame for plotting'):
        if df_temp.loc[i, 'num_of_pairs'] >= 1:
            # get pedestrian info as dictionary
            values_i = df_temp.loc[i, ['recordingId', 'trackId', 'xCenter', 
                                       'yCenter','class', 'frame']].values
            values_i_dict = dict(zip(keys_i, values_i))
            # print('values_i_dict:', values_i_dict)
            
            # get pairs info from the pairs column using eval
            object_j_pairs_list = eval( df_temp.loc[i, 'pairs'], {"inf": np.inf})
            # print('object_j_pairs_list: ',object_j_pairs_list, 'type:', type(object_j_pairs_list))
            
            # iterate over peers j_n of pedestrian i
            # print('total number of pairs:', df_temp.loc[i, 'num_of_pairs'])
            for index, (peer_row, peer_object_j_class, ttc_value) in enumerate(object_j_pairs_list): #index is the peer_number 
                # print('index of peer in pairs:', index)
                # create peer dictionary
                values_j_n = df_temp.loc[peer_row, ['trackId','xCenter', 'yCenter','class', 'frame']].values
                values_j_n_dict = dict(zip(keys_j_n, values_j_n))
                values_j_n_dict['ttc'] = ttc_value
                # print('values_j_n_dict:', values_j_n_dict) 
                # combine and append to a df row 
                combined_dict = {**values_i_dict, **values_j_n_dict} # combine i and n_j dictionaries
                df_ttc = pd.concat([df_ttc, pd.DataFrame([combined_dict])], ignore_index=True) # put in df
            # print()
    return df_ttc

In [9]:
root_dir

'C:\\Users\\yftac\\Documents\\00_Project_Afeka\\ttc_calculation'

create folder to save csv's

In [10]:
os.chdir(root_dir)
# create folder to save files 
folder_name='08_data_with_ttc_for_plotting'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    print('folder created')
# change to working dir
os.chdir(folder_name)

folder created


In [11]:
def create_df_ttc_for_plot_and_save(x, dfs, recordingId_list, keys_i, keys_j_n):
    df_temp = dfs[x].copy()
    # create new df for plotting
    df_ttc_for_plot = create_df_series_for_plot(df_temp, df_ttc, keys_i, keys_j_n)
    # save the new df
    df_ttc_for_plot.to_csv(f'{recordingId_list[x]}_ttc_for_plotting.csv', index=False)
    print('df saved:', f'{recordingId_list[x]}_ttc_for_plotting.csv')
    print('saved in folder:', os.getcwd())
    return

# x=7
# print('df name:',recordingId_list[x])
# df_temp = dfs[x]
# df_ttc = create_df_series_for_plot(df_temp, df_ttc, keys_i, keys_j_n)
# df_ttc.to_csv(f'{recordingId_list[x]}_ttc_for_plotting.csv', index=False)
# os.chdir('..')

In [12]:
create_df_ttc_for_plot_and_save(0, dfs, recordingId_list, keys_i, keys_j_n)

Creating DataFrame for plotting: 100%|██████████| 42378/42378 [09:30<00:00, 74.24it/s]  


df saved: 18_ttc_for_plotting.csv
saved in folder: C:\Users\yftac\Documents\00_Project_Afeka\ttc_calculation\08_data_with_ttc_for_plotting


In [13]:
create_df_ttc_for_plot_and_save(1, dfs, recordingId_list, keys_i, keys_j_n)

Creating DataFrame for plotting: 100%|██████████| 33777/33777 [03:04<00:00, 183.46it/s] 

df saved: 19_ttc_for_plotting.csv
saved in folder: C:\Users\yftac\Documents\00_Project_Afeka\ttc_calculation\08_data_with_ttc_for_plotting


In [14]:
create_df_ttc_for_plot_and_save(2, dfs, recordingId_list, keys_i, keys_j_n)

Creating DataFrame for plotting: 100%|██████████| 43601/43601 [06:38<00:00, 109.37it/s] 


df saved: 20_ttc_for_plotting.csv
saved in folder: C:\Users\yftac\Documents\00_Project_Afeka\ttc_calculation\08_data_with_ttc_for_plotting


In [15]:
create_df_ttc_for_plot_and_save(3, dfs, recordingId_list, keys_i, keys_j_n)

Creating DataFrame for plotting: 100%|██████████| 54919/54919 [16:58<00:00, 53.90it/s]  


df saved: 21_ttc_for_plotting.csv
saved in folder: C:\Users\yftac\Documents\00_Project_Afeka\ttc_calculation\08_data_with_ttc_for_plotting


In [16]:
create_df_ttc_for_plot_and_save(4, dfs, recordingId_list, keys_i, keys_j_n)

Creating DataFrame for plotting: 100%|██████████| 56011/56011 [14:44<00:00, 63.33it/s]  


df saved: 22_ttc_for_plotting.csv
saved in folder: C:\Users\yftac\Documents\00_Project_Afeka\ttc_calculation\08_data_with_ttc_for_plotting


In [17]:
create_df_ttc_for_plot_and_save(5, dfs, recordingId_list, keys_i, keys_j_n)

Creating DataFrame for plotting:   0%|          | 0/53853 [00:00<?, ?it/s]

Creating DataFrame for plotting: 100%|██████████| 53853/53853 [11:50<00:00, 75.84it/s]  


df saved: 23_ttc_for_plotting.csv
saved in folder: C:\Users\yftac\Documents\00_Project_Afeka\ttc_calculation\08_data_with_ttc_for_plotting


In [18]:
create_df_ttc_for_plot_and_save(6, dfs, recordingId_list, keys_i, keys_j_n)

Creating DataFrame for plotting: 100%|██████████| 44711/44711 [10:15<00:00, 72.59it/s]  


df saved: 24_ttc_for_plotting.csv
saved in folder: C:\Users\yftac\Documents\00_Project_Afeka\ttc_calculation\08_data_with_ttc_for_plotting


In [19]:
create_df_ttc_for_plot_and_save(7, dfs, recordingId_list, keys_i, keys_j_n)

Creating DataFrame for plotting: 100%|██████████| 55189/55189 [11:15<00:00, 81.76it/s]  


df saved: 25_ttc_for_plotting.csv
saved in folder: C:\Users\yftac\Documents\00_Project_Afeka\ttc_calculation\08_data_with_ttc_for_plotting


In [21]:
create_df_ttc_for_plot_and_save(8, dfs, recordingId_list, keys_i, keys_j_n)

Creating DataFrame for plotting: 100%|██████████| 60034/60034 [19:37<00:00, 50.96it/s]  


df saved: 26_ttc_for_plotting.csv
saved in folder: C:\Users\yftac\Documents\00_Project_Afeka\ttc_calculation\08_data_with_ttc_for_plotting


In [22]:
create_df_ttc_for_plot_and_save(9, dfs, recordingId_list, keys_i, keys_j_n)

Creating DataFrame for plotting: 100%|██████████| 65115/65115 [14:17<00:00, 75.94it/s]  


df saved: 27_ttc_for_plotting.csv
saved in folder: C:\Users\yftac\Documents\00_Project_Afeka\ttc_calculation\08_data_with_ttc_for_plotting


In [23]:
create_df_ttc_for_plot_and_save(10, dfs, recordingId_list, keys_i, keys_j_n)

Creating DataFrame for plotting: 100%|██████████| 56613/56613 [10:49<00:00, 87.16it/s]  


df saved: 28_ttc_for_plotting.csv
saved in folder: C:\Users\yftac\Documents\00_Project_Afeka\ttc_calculation\08_data_with_ttc_for_plotting


In [24]:
create_df_ttc_for_plot_and_save(11, dfs, recordingId_list, keys_i, keys_j_n)

Creating DataFrame for plotting: 100%|██████████| 67337/67337 [19:50<00:00, 56.57it/s]  


df saved: 29_ttc_for_plotting.csv
saved in folder: C:\Users\yftac\Documents\00_Project_Afeka\ttc_calculation\08_data_with_ttc_for_plotting
